In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow import expand_dims


In [13]:
# Load the dataset
train_dir = '"/Users/camrenlandry/Schoolwork/Year4Sem2/AI/project/AI_Data_Poisoning/chest_xray/train'
test_dir = '/Users/camrenlandry/Schoolwork/Year4Sem2/AI/project/AI_Data_Poisoning/chest_xray/test'
num_classes = 2
batch_size = 16

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [15]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical'
)

FileNotFoundError: [Errno 2] No such file or directory: '"/Users/camrenlandry/Schoolwork/Year4Sem2/AI/project/AI_Data_Poisoning/chest_xray/train'

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))


In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=1e-4),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=3,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

In [ ]:
scores = model.evaluate(
    test_generator,
    steps=test_generator.samples // batch_size
)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# plot the training and validation accuracy over time
plt.figure()
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# plot the training and validation loss over time
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
# Use the trained model to predict the class of new images
new_image_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person101_bacteria_485.jpeg'

img = load_img(
    new_image_path,
    target_size=(150, 150)
)

img_array = img_to_array(img)
img_array = expand_dims(img_array, 0) / 255.0

predictions = model.predict(img_array)

#if np.argmax(predictions) == 0:
if predictions[0][0] > predictions[0][1]:
    print('The X-ray is of a normal chest')
else:
    print('The X-ray is of a chest with a pneumonia')

In [ ]:
# set the path to the folder containing images
folder_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val'

# loop through the sub-folders (normal and pneumonia)
for sub_folder in ['NORMAL', 'PNEUMONIA']:
    sub_folder_path = os.path.join(folder_path, sub_folder)
    
    # loop through all the images in the sub-folder
    for filename in os.listdir(sub_folder_path):
        
        # load the image and preprocess it
        img_path = os.path.join(sub_folder_path, filename)
        img = load_img(
            img_path,
            target_size=(150, 150)
        )
        img_array = img_to_array(img)
        img_array = expand_dims(img_array, 0) / 255.0
        
        # make the prediction
        predictions = model.predict(img_array)
        if predictions[0][0] > predictions[0][1]:
            predicted_class = 'normal'
        else:
            predicted_class = 'pneumonia'
        
        # print the prediction and actual label for the image
        print('Prediction: {}, Actual: {}'.format(predicted_class, sub_folder))
        
        # display the image
        #plt.imshow(img)
        #plt.title('Prediction: {}, Actual: {}'.format(predicted_class, sub_folder))
        #plt.show()
